# Minesweeper Version 2
## Library

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import default_rng
from warnings import warn
%reload_ext blackcellmagic

## Define Minesweeper

In [80]:
size, mines = np.array([15, 10]), 16

field = np.zeros(size[0]*size[1])
rng = default_rng()
numbers = rng.choice(np.product(size), size=mines, replace=False)
field[numbers] = -9
field = field.reshape(size)
minefield = np.zeros(size+2) + float("inf")
minefield[1:-1, 1:-1] = field
positions = np.array(np.where(field==-9))+1
for x, y in zip(*positions):
    grid = minefield[x-1:x+2, y-1:y+2]
    grid[grid != -9] -= 1
minefield[minefield == 0] = -float("inf")
temp = np.ones_like(field) * mines/np.product(size)
prob = np.zeros(size+2) + float("inf")
prob[1:-1, 1:-1] = temp

## Basic Functions

In [87]:
def pick(field, locations, prob):
    x,y = locations
    if abs(field[x,y]) != float("inf"):
        field[x,y] = abs(field[x,y])
        if field[x, y] == 9:
            warn("Mine hit")
    elif field[x, y] == -float("inf"):
        for i in range(-1, 2):
            for j in range(-1, 2):
                pick(field, (x+i, y+j), prob)
    prob[np.logical_and(minefield > 0, minefield < float("inf"))] = 0

def quickPick(field):
    


(1, 3)


In [91]:
float("inf")-float("inf") < float("inf")

False